# Knet RNN example

In [1]:
# After installing and starting Julia run the following to install the required packages:
# julia> Pkg.init(); for p in ("CUDAdrv","IJulia","Knet","PyCall","JLD2"); Pkg.add(p); end

In [2]:
Pkg.checkout("Knet","ilkarman") # make sure we have the right Knet version
Pkg.build("Knet")
using Knet
True=true # so we can read the python params
include("common/params_lstm.py");

INFO: Checking out Knet ilkarman...
INFO: Pulling Knet latest ilkarman...
INFO: No packages to install, update or remove
INFO: Building Knet


make: 'libknet8.so' is up to date.


In [3]:
println("OS: ", Sys.KERNEL)
println("Julia: ", VERSION)
println("Knet: ", Pkg.installed("Knet"))
println("GPU: ", readstring(`nvidia-smi --query-gpu=name --format=csv,noheader`))

OS: Linux
Julia: 0.6.1
Knet: 0.8.5+
GPU: Tesla K80



In [4]:
# define model
function initmodel()
    rnnSpec,rnnWeights = rnninit(EMBEDSIZE,NUMHIDDEN; rnnType=:gru)
    inputMatrix = KnetArray(xavier(Float32,EMBEDSIZE,MAXFEATURES))
    outputMatrix = KnetArray(xavier(Float32,2,NUMHIDDEN))
    return rnnSpec,(rnnWeights,inputMatrix,outputMatrix)
end;

In [5]:
# define loss and its gradient
function predict(weights, inputs, rnnSpec)
    rnnWeights, inputMatrix, outputMatrix = weights # (1,1,W), (X,V), (2,H)
    indices = hcat(inputs...)' # (B,T)
    rnnInput = inputMatrix[:,indices] # (X,B,T)
    rnnOutput = rnnforw(rnnSpec, rnnWeights, rnnInput)[1] # (H,B,T)
    return outputMatrix * rnnOutput[:,:,end] # (2,H) * (H,B) = (2,B)
end

loss(w,x,y,r)=nll(predict(w,x,r),y)
lossgradient = grad(loss);

In [6]:
# load data
include(Knet.dir("data","imdb.jl"))
@time (xtrn,ytrn,xtst,ytst,imdbdict)=imdb(maxlen=MAXLEN,maxval=MAXFEATURES)
for d in (xtrn,ytrn,xtst,ytst); println(summary(d)); end

INFO: Loading IMDB...


  9.804228 seconds (15.66 M allocations: 821.100 MiB, 4.36% gc time)
25000-element Array{Array{Int32,1},1}
25000-element Array{Int8,1}
25000-element Array{Array{Int32,1},1}
25000-element Array{Int8,1}


In [7]:
# prepare for training
weights = nothing; knetgc(); # Reclaim memory from previous run
rnnSpec,weights = initmodel()
optim = optimizers(weights, Adam; lr=LR, beta1=BETA_1, beta2=BETA_2, eps=EPS);

In [8]:
# force precompile (optional)
(x,y) = first(minibatch(xtrn,ytrn,BATCHSIZE))
@time lossgradient(weights,x,y,rnnSpec);

  3.525070 seconds (1.49 M allocations: 80.569 MiB, 0.89% gc time)


In [9]:
# 30s
info("Training...")
@time for epoch in 1:EPOCHS
    @time for (x,y) in minibatch(xtrn,ytrn,BATCHSIZE;shuffle=true)
        grads = lossgradient(weights,x,y,rnnSpec)
        update!(weights, grads, optim)
    end
end

INFO: Training...


 10.559463 seconds (668.98 k allocations: 61.830 MiB, 4.87% gc time)
  9.602975 seconds (358.03 k allocations: 44.377 MiB, 6.43% gc time)
  9.590503 seconds (358.76 k allocations: 44.389 MiB, 6.38% gc time)
 29.754309 seconds (1.39 M allocations: 150.835 MiB, 5.86% gc time)


In [10]:
info("Testing...")
total = correct = 0
@time for (x,y) in minibatch(xtst,ytst,BATCHSIZE)
    total += accuracy(predict(weights,x,rnnSpec), y; average=false)
    correct += length(y)
end
total/correct

INFO: Testing...


  3.055123 seconds (289.27 k allocations: 45.848 MiB, 4.54% gc time)


0.8529647435897436